# Find Start and End Date for each consecutive event status
### The objective of the code challenge is order the events by event date and then get the start date and end date between each event change.<br>


In [0]:
from pyspark.sql.functions import to_date, col, lag, lead, when, sum, first, last
from pyspark.sql.window import Window

In [0]:
data = [('win','2023-10-01'),('win','2023-10-02'),('win','2023-10-03'),('win','2023-10-04'),('loss','2023-10-05'),('loss','2023-10-06'),('loss','2023-10-07'),('win','2023-10-08'),('win','2023-10-09'),('win','2023-10-10'),('win','2023-10-11'),('win','2023-10-12'),('loss','2023-10-13')]
df = spark.createDataFrame(data,['event','event_date'])
display(df)

event,event_date
win,2023-10-01
win,2023-10-02
win,2023-10-03
win,2023-10-04
loss,2023-10-05
loss,2023-10-06
loss,2023-10-07
win,2023-10-08
win,2023-10-09
win,2023-10-10


In [0]:
df = df.withColumn('event_date',to_date(col('event_date')))
display(df)

event,event_date
win,2023-10-01
win,2023-10-02
win,2023-10-03
win,2023-10-04
loss,2023-10-05
loss,2023-10-06
loss,2023-10-07
win,2023-10-08
win,2023-10-09
win,2023-10-10


In [0]:
df_event_change = df.withColumn('event_change', when(col('event') != lag('event').over(Window.orderBy('event_date')),1).otherwise(0))
df_event_change.display()

event,event_date,event_change
win,2023-10-01,0
win,2023-10-02,0
win,2023-10-03,0
win,2023-10-04,0
loss,2023-10-05,1
loss,2023-10-06,0
loss,2023-10-07,0
win,2023-10-08,1
win,2023-10-09,0
win,2023-10-10,0


In [0]:
df_event_change = df_event_change.withColumn('event_group', sum('event_change').over(Window.orderBy('event_date')))
df_event_change.display()

event,event_date,event_change,event_group
win,2023-10-01,0,0
win,2023-10-02,0,0
win,2023-10-03,0,0
win,2023-10-04,0,0
loss,2023-10-05,1,1
loss,2023-10-06,0,1
loss,2023-10-07,0,1
win,2023-10-08,1,2
win,2023-10-09,0,2
win,2023-10-10,0,2


In [0]:
df_event_result = df_event_change.groupBy('event_group','event').agg(first('event_date').alias('start_date'),last('event_date').alias('end_date')).drop('event_group')

df_event_result.display()

event,start_date,end_date
win,2023-10-01,2023-10-04
loss,2023-10-05,2023-10-07
win,2023-10-08,2023-10-12
loss,2023-10-13,2023-10-13
